In [1]:
class Literal:
    
    def __init__(self, name):
        if(name[0] == '-'):
            self.name = name[1:]
            self.sign = 'negative'
        else:
            self.name = name
            self.sign = 'positive'
        
    def __neg__(self):
        return Literal('-'+self.name)
    
    def __eq__(self, other):
        return self.name == other.name
        
A = Literal('A')
B = Literal('B')
C = Literal('C')
D = Literal('D')
E = Literal('E')
F = Literal('F')

KB = [[-A, B, E], [-B, A] , [-E, A], [-E, D], [-C, -F, -B], [-E, B], [-B, F], [-B,C]]

In [2]:
import copy
import collections

model = []

# DPLL_Satisfiable requires as input a KB
# it processes it so it converts the Literal objects to integers
# that follow the rule map(-Literal) = -map(Literal) and
# map(Literal) = map(OtherLiteral) if and only if Literal = OtherLiteral
def DPLL_Satisfiable(KB):
    flat_KB = [symbol for clause in KB for symbol in clause]
    d = {}
    for i in range(1, len(flat_KB) + 1):
        d[flat_KB[i - 1].name] = i
        
    inv_d = {v: k for k, v in d.items()}
    
    int_KB = []

    for i in KB:
        clause = []
        for j in i:
            if(j.sign == 'positive'):
                clause.append(d[j.name])
            else:
                clause.append(-d[j.name])
        int_KB.append(clause)
    return (DPLL(int_KB), model_transform(model, inv_d))


# DPLL does what we know it does, here with the unit clause
# and degree heuristics.
# The deepcopies are used as to bypass scope issues that would
# otherwise arise by calling simplify over the same KB sequentially
def DPLL(KB):
    KB1 = copy.deepcopy(KB)
    KB2 = copy.deepcopy(KB)
    KB5 = copy.deepcopy(KB)
    KB6 = copy.deepcopy(KB)
    
    if (len(KB) == 0):
        return True
    for i in KB:
        
        #beginning of unit clause heuristic
        if (len(i) == 1):
            if (DPLL(simplify(KB5, i[0]))):
                model.append(i[0])
                return True
        #end of unit clause heuristic
        
        if (len(i) == 0):
            return False
    
    
    #begining of pure symbol heuristic (choosing pure symbol that occurs the most)
    lst_purity = [symbol for clause in KB for symbol in clause]
    counts_purity = collections.Counter(lst_purity)

    sorted_by_frequency_purity = sorted(lst_purity, key=lambda x: -counts_purity[x])
    for i in sorted_by_frequency_purity:
        if -i not in sorted_by_frequency_purity:
            P_pure = i
            break
    
    if 'P_pure' in locals():
    
        if (DPLL(simplify(KB6, P_pure))):
            model.append(P_pure)
            return True
    #end of pure symbol heuristic (choosing pure symbol that occurs the most)
    
    
    #beginning of degree heuristic    
    lst = [abs(symbol) for clause in KB for symbol in clause]
    counts = collections.Counter(lst)
    sorted_by_frequency = sorted(lst, key=lambda x: -counts[x])
    P = sorted_by_frequency[0]
    #end of degree heuristic
    
    KB3 = simplify(KB1, P)
    KB4 = simplify(KB2, -P)
    if (DPLL(simplify(KB3, P))):
        model.append(P)
        return True
    elif (DPLL(simplify(KB4, -P))):
        model.append(-P)
        return True
    else:
        return False
       
# simplify returns clauses conjuncted with literal = True
# so clauses where literal appears are deleted and
# -literal is deleted from the clauses where it appears
def simplify(clauses, literal):
    remove_clauses = []
    new_clauses = clauses[:]
    for i in range(len(clauses)):
        if literal in clauses[i]:
            remove_clauses.append(clauses[i])
        if -literal in clauses[i]:
            new_clauses[i].remove(-literal)
    for i in remove_clauses:       
        new_clauses.remove(i)
    return new_clauses

# model_transform is in charge of converting integers to Booleans
def model_transform(model, dictionary):
    transform = {}
    for i in model:
        if i > 0:
            transform[dictionary[i]] = True
        if i < 0:
            transform[dictionary[-1 * i]] = False
            
    for i in list(dictionary.values()):
        if i not in list(transform.keys()):
            transform[i] = "free"
    return transform
        



### Test

In [3]:
print(DPLL_Satisfiable(KB))

(True, {'A': False, 'E': False, 'B': False, 'D': True, 'C': 'free', 'F': 'free'})
